# Node

In [ ]:
!pip list

In [ ]:
import time
import torch as th
th.set_num_threads(1)
import syft as sy
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient
hook = sy.TorchHook(th)
alice = DataCentricFLClient(hook, "ws://localhost:7600")
bob = DataCentricFLClient(hook, "ws://localhost:7601")
my_grid = sy.PrivateGridNetwork(alice,bob)
payload = 1_000 # in Mo
t = th.rand(1, int(payload * 10**6 / 4)) # 4 because torch float32 are encded over 4 bytes
timer = time.time()
t.send(alice)
delay = time.time() - timer
print('bandwidth', payload/delay)

In [ ]:
print(type(alice.ws))

In [ ]:
alice.ws.send("Hello")

In [ ]:
# alice.ws.recv()
import pyarrow as pa
data = [

    pa.array([1, 2, 3, 4]),

    pa.array(['foo', 'bar', 'baz', None]),

    pa.array([True, None, False, True])

]
batch = pa.record_batch(data, names=['f0', 'f1', 'f2'])
print(batch)

In [ ]:
# sink = alice.ws
# writer = pa.ipc.new_stream(sink, batch.schema)

In [5]:
import argparse
import ast
import threading
import time

import pyarrow
import pyarrow as pa
import pyarrow.flight
import pyarrow.csv as csv
import numpy as np
import struct

class FlightServer(pyarrow.flight.FlightServerBase):
    def __init__(self, host="localhost", location=None,
                 tls_certificates=None, verify_client=False,
                 root_certificates=None, auth_handler=None):
        super(FlightServer, self).__init__(
            location, auth_handler, tls_certificates, verify_client,
            root_certificates)
        self.flights = {}
        self.host = host
        self.tls_certificates = tls_certificates

    @classmethod
    def descriptor_to_key(self, descriptor):
        return (descriptor.descriptor_type.value, descriptor.command,
                tuple(descriptor.path or tuple()))

    def _make_flight_info(self, key, descriptor, table):
        if self.tls_certificates:
            location = pyarrow.flight.Location.for_grpc_tls(
                self.host, self.port)
        else:
            location = pyarrow.flight.Location.for_grpc_tcp(
                self.host, self.port)
        endpoints = [pyarrow.flight.FlightEndpoint(repr(key), [location]), ]

        mock_sink = pyarrow.MockOutputStream()
        stream_writer = pyarrow.RecordBatchStreamWriter(
            mock_sink, table.schema)
        stream_writer.write_table(table)
        stream_writer.close()
        data_size = mock_sink.size()

        return pyarrow.flight.FlightInfo(table.schema,
                                         descriptor, endpoints,
                                         table.num_rows, data_size)

    def list_flights(self, context, criteria):
        for key, table in self.flights.items():
            if key[1] is not None:
                descriptor = \
                    pyarrow.flight.FlightDescriptor.for_command(key[1])
            else:
                descriptor = pyarrow.flight.FlightDescriptor.for_path(*key[2])

            yield self._make_flight_info(key, descriptor, table)

    def get_flight_info(self, context, descriptor):
        key = FlightServer.descriptor_to_key(descriptor)
        if key in self.flights:
            table = self.flights[key]
            return self._make_flight_info(key, descriptor, table)
        raise KeyError('Flight not found.')

    def do_put(self, context, descriptor, reader, writer):
        key = FlightServer.descriptor_to_key(descriptor)
        t = time.time()
        print(f"Got a new key: {key}")
        
        print(type(reader))

#         table = reader.read_all()
        record_batch = reader.read_chunk().data
        # self.flights[key] = reader.read_all()
        dl_time = time.time() - t
        
        print(f"Original object received in {dl_time}:")
        t = time.time()
#         print(table.to_batches()[0])
        print(record_batch)
        print(pa.deserialize(record_batch[0].buffers()[2]))

        
#         print(pa.deserialize(table.to_batches()[0][0].buffers()[2]))
        
#         print(f"Deserialized in {time.time() - t}")
        
#         record_batch = pa.RecordBatch.from_arrays([pa.array([pa.serialize(np.array([42])).to_buffer().to_pybytes()])], names=[""])
#         buf_response = pa.serialize(np.array([42])).to_buffer()
        print(struct.pack('<i', 42))
        print(type(struct.pack('<i', 42)))
        writer.write(struct.pack('<i', 42))
        print("Written response")
#         writer.write(buf_response)
#         writer.close()
        
        
#         print("Read all:")
        
#         print(table)
#         print(type(table))
        
#         b = table.to_batches()
#         print(b)
# #         print(table.to_pydict())
#         print(b[0])
#         print(b[0].column(0))
#         c = b[0].column(0)
#         print(pa.deserialize(c))
        
#         # print(self.flights[key])
#         print(table.to_pandas())
#         print(table.shape)
#         print(f"Time: {dl_time}")
        
    def back_to_bin():
        return


    def do_get(self, context, ticket):
        key = ast.literal_eval(ticket.ticket.decode())
        if key not in self.flights:
            return None
        return pyarrow.flight.RecordBatchStream(self.flights[key])

    def list_actions(self, context):
        return [
            ("clear", "Clear the stored flights."),
            ("shutdown", "Shut down this server."),
        ]

    def do_action(self, context, action):
        if action.type == "clear":
            raise NotImplementedError(
                "{} is not implemented.".format(action.type))
        elif action.type == "healthcheck":
            pass
        elif action.type == "shutdown":
            yield pyarrow.flight.Result(pyarrow.py_buffer(b'Shutdown!'))
            # Shut down on background thread to avoid blocking current
            # request
            threading.Thread(target=self._shutdown).start()
        else:
            raise KeyError("Unknown action {!r}".format(action.type))

    def _shutdown(self):
        """Shut down after a delay."""
        print("Server is shutting down...")
        time.sleep(2)
        self.shutdown()


In [ ]:
tls_certificates = []
scheme = "grpc+tcp"
host="localhost"
port="7604"

location = "{}://{}:{}".format(scheme, host, port)

server = FlightServer(host, location)

print("Serving on", location)
server.serve()

Serving on grpc+tcp://localhost:7604
Got a new key: (2, b'feed_crypto_store_fssb', ())
<class 'pyarrow._flight.MetadataRecordBatchReader'>
Original object received in 0.0010600090026855469:
pyarrow.RecordBatch
: binary
[[189  97 145 ... 101  36 110]]
b'*\x00\x00\x00'
<class 'bytes'>
Written response
Got a new key: (2, b'feed_crypto_store_fssb', ())
<class 'pyarrow._flight.MetadataRecordBatchReader'>
Original object received in 0.0014543533325195312:
pyarrow.RecordBatch
: binary
[[189  97 145 ... 101  36 110]]
b'*\x00\x00\x00'
<class 'bytes'>
Written response


In [ ]:
b = 

## IPC